In [ ]:
import requests
import pandas as pd
import numpy as np
from colorama import init, Fore, Style
import time

init(autoreset=True)

BASE_URL = "https://api.binance.com/api/v3/klines"

# Timeframes
timeframes = {
    "15m": "15m",
    "1h": "1h",
    "4h": "4h"
}

# Fetch USDT pairs
def get_usdt_pairs():
    url = "https://api.binance.com/api/v3/exchangeInfo"
    data = requests.get(url).json()
    return [s['symbol'] for s in data['symbols'] if s['quoteAsset'] == "USDT" and s['status'] == "TRADING"]

# Fetch OHLCV data
def get_klines(symbol, interval="15m", limit=100):
    try:
        url = f"{BASE_URL}?symbol={symbol}&interval={interval}&limit={limit}"
        data = requests.get(url).json()
        df = pd.DataFrame(data, columns=[
            "time","open","high","low","close","volume",
            "close_time","qav","num_trades","tbbav","tbqav","ignore"
        ])
        df["open"] = df["open"].astype(float)
        df["high"] = df["high"].astype(float)
        df["low"] = df["low"].astype(float)
        df["close"] = df["close"].astype(float)
        df["volume"] = df["volume"].astype(float)

        # Indicators
        df["ema7"] = df["close"].ewm(span=7).mean()
        df["sma21"] = df["close"].rolling(window=21).mean()
        df["rsi"] = compute_rsi(df["close"], 14)

        return df
    except Exception as e:
        print(f"⚠️ get_klines error {symbol}: {e}")
        return None

# RSI
def compute_rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=period-1, adjust=False).mean()
    ema_down = down.ewm(com=period-1, adjust=False).mean()
    rs = ema_up / ema_down
    return 100 - (100 / (1 + rs))

# Support & Resistance
def find_support_resistance(df):
    closes = df["close"].values
    support = np.min(closes[-20:])
    resistance = np.max(closes[-20:])
    return support, resistance

# Apply strategy (candlestick + ema/sma + rsi)
def apply_strategy(df):
    close = df["close"].iloc[-1]
    volume = df["volume"].iloc[-1]
    prev_volume = df["volume"].iloc[-2]
    ema = df["ema7"].iloc[-1]
    sma = df["sma21"].iloc[-1]
    rsi = df["rsi"].iloc[-1]
    support, resistance = find_support_resistance(df)

    patterns = []
    body = abs(df["close"].iloc[-1] - df["open"].iloc[-1])
    range_ = df["high"].iloc[-1] - df["low"].iloc[-1]
    if range_ > 0 and body / range_ < 0.1:
        patterns.append("Doji")

    return {
        "price": close,
        "volume": volume,
        "prev_volume": prev_volume,
        "ema": ema,
        "sma": sma,
        "rsi": rsi,
        "support": support,
        "resistance": resistance,
        "patterns": patterns
    }

# Run scan
def run_signal_scan():
    usdt_pairs = get_usdt_pairs()
    found = 0

    for symbol in usdt_pairs[:30]:  # Limit to 30 coins
        for tf_name, tf_interval in timeframes.items():
            try:
                df = get_klines(symbol, interval=tf_interval, limit=50)
                if df is None or len(df) < 25:
                    continue

                signal = apply_strategy(df)
                if signal:
                    arrow = "🔼" if signal['volume'] > signal['prev_volume'] else "🔽"
                    color_arrow = f"{Fore.GREEN}{arrow}" if signal['volume'] > signal['prev_volume'] else f"{Fore.RED}{arrow}"

                    print(
                        f"{symbol:<12} {tf_name:<5} {', '.join(signal['patterns']):<25} "
                        f"{signal['price']:<12.4f} {str(signal['volume']) + color_arrow:<12} "
                        f"{signal['rsi']:<10.4f} {signal['ema']:<12.4f} {signal['sma']:<12.4f} "
                        f"{signal['support']:<12.4f} {signal['resistance']:<12.4f}"
                    )
                    found += 1
            except Exception as e:
                print(f"⚠️ Error for {symbol} at {tf_name} timeframe: {e}")
                continue
            time.sleep(0.05)

    if found == 0:
        print("No signals found this round.")


if __name__ == "__main__":
    while True:
        run_signal_scan()
        time.sleep(5)   # refresh every 5 seconds


In [1]:
import requests
import pandas as pd
import numpy as np
from colorama import init, Fore, Style
import time

init(autoreset=True)

BASE_URL = "https://api.binance.com/api/v3/klines"

# 🔹 Apna Discord webhook yahan daalo
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/XXXX/XXXX"

# Timeframes
timeframes = {
    "15m": "15m",
    "1h": "1h",
    "4h": "4h"
}

# Discord par message bhejne ka function
def send_discord_message(message):
    try:
        data = {"content": message}
        response = requests.post(DISCORD_WEBHOOK_URL, json=data)
        if response.status_code == 204:
            print("✅ Discord par message bheja gaya!")
        else:
            print(f"❌ Discord Error: {response.status_code}, {response.text}")
    except Exception as e:
        print(f"⚠️ Discord error: {e}")

# Fetch USDT pairs
def get_usdt_pairs():
    url = "https://api.binance.com/api/v3/exchangeInfo"
    data = requests.get(url).json()
    return [s['symbol'] for s in data['symbols'] if s['quoteAsset'] == "USDT" and s['status'] == "TRADING"]

# Fetch OHLCV data
def get_klines(symbol, interval="15m", limit=100):
    try:
        url = f"{BASE_URL}?symbol={symbol}&interval={interval}&limit={limit}"
        data = requests.get(url).json()
        df = pd.DataFrame(data, columns=[
            "time","open","high","low","close","volume",
            "close_time","qav","num_trades","tbbav","tbqav","ignore"
        ])
        df["open"] = df["open"].astype(float)
        df["high"] = df["high"].astype(float)
        df["low"] = df["low"].astype(float)
        df["close"] = df["close"].astype(float)
        df["volume"] = df["volume"].astype(float)

        # Indicators
        df["ema7"] = df["close"].ewm(span=7).mean()
        df["sma21"] = df["close"].rolling(window=21).mean()
        df["rsi"] = compute_rsi(df["close"], 14)

        return df
    except Exception as e:
        print(f"⚠️ get_klines error {symbol}: {e}")
        return None

# RSI
def compute_rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=period-1, adjust=False).mean()
    ema_down = down.ewm(com=period-1, adjust=False).mean()
    rs = ema_up / ema_down
    return 100 - (100 / (1 + rs))

# Support & Resistance
def find_support_resistance(df):
    closes = df["close"].values
    support = np.min(closes[-20:])
    resistance = np.max(closes[-20:])
    return support, resistance

# Apply strategy (candlestick + ema/sma + rsi)
def apply_strategy(df):
    close = df["close"].iloc[-1]
    volume = df["volume"].iloc[-1]
    prev_volume = df["volume"].iloc[-2]
    ema = df["ema7"].iloc[-1]
    sma = df["sma21"].iloc[-1]
    rsi = df["rsi"].iloc[-1]
    support, resistance = find_support_resistance(df)

    patterns = []
    body = abs(df["close"].iloc[-1] - df["open"].iloc[-1])
    range_ = df["high"].iloc[-1] - df["low"].iloc[-1]
    if range_ > 0 and body / range_ < 0.1:
        patterns.append("Doji")

    return {
        "price": close,
        "volume": volume,
        "prev_volume": prev_volume,
        "ema": ema,
        "sma": sma,
        "rsi": rsi,
        "support": support,
        "resistance": resistance,
        "patterns": patterns
    }

# Run scan
def run_signal_scan():
    usdt_pairs = get_usdt_pairs()
    found = 0

    for symbol in usdt_pairs[:30]:  # Limit to 30 coins
        for tf_name, tf_interval in timeframes.items():
            try:
                df = get_klines(symbol, interval=tf_interval, limit=50)
                if df is None or len(df) < 25:
                    continue

                signal = apply_strategy(df)
                if signal:
                    arrow = "🔼" if signal['volume'] > signal['prev_volume'] else "🔽"

                    msg = (
                        f"**{symbol}** | {tf_name}\n"
                        f"Pattern: {', '.join(signal['patterns']) if signal['patterns'] else 'None'}\n"
                        f"Price: {signal['price']:.4f}\n"
                        f"Volume: {signal['volume']} {arrow}\n"
                        f"RSI: {signal['rsi']:.2f}\n"
                        f"EMA(7): {signal['ema']:.4f} | SMA(21): {signal['sma']:.4f}\n"
                        f"Support: {signal['support']:.4f} | Resistance: {signal['resistance']:.4f}"
                    )

                    print(msg)                # Console output
                    send_discord_message(msg) # Discord alert
                    found += 1
            except Exception as e:
                print(f"⚠️ Error for {symbol} at {tf_name} timeframe: {e}")
                continue
            time.sleep(0.05)

    if found == 0:
        print("No signals found this round.")


if __name__ == "__main__":
    while True:
        run_signal_scan()
        time.sleep(5)   # refresh every 5 seconds


**BTCUSDT** | 15m
Pattern: None
Price: 108084.0100
Volume: 59.65866 🔽
RSI: 56.45
EMA(7): 107871.1972 | SMA(21): 107728.4129
Support: 107309.9100 | Resistance: 108150.2400
❌ Discord Error: 400, {"webhook_id": ["Value \"XXXX\" is not snowflake."]}
**BTCUSDT** | 1h
Pattern: None
Price: 108084.0000
Volume: 291.86883 🔽
RSI: 44.01
EMA(7): 107968.2181 | SMA(21): 108441.3090
Support: 107409.1000 | Resistance: 109146.6200
❌ Discord Error: 400, {"webhook_id": ["Value \"XXXX\" is not snowflake."]}
**BTCUSDT** | 4h
Pattern: None
Price: 108084.0000
Volume: 1485.50637 🔽
RSI: 37.38
EMA(7): 108340.0660 | SMA(21): 109027.6657
Support: 107660.0100 | Resistance: 111704.2300
❌ Discord Error: 400, {"webhook_id": ["Value \"XXXX\" is not snowflake."]}
**ETHUSDT** | 15m
Pattern: None
Price: 4406.2900
Volume: 3586.2108 🔼
RSI: 52.25
EMA(7): 4396.4888 | SMA(21): 4394.2957
Support: 4374.2100 | Resistance: 4417.8200
❌ Discord Error: 400, {"webhook_id": ["Value \"XXXX\" is not snowflake."]}
**ETHUSDT** | 1h
Pattern

KeyboardInterrupt: 

In [3]:
import requests
import pandas as pd
import numpy as np
from colorama import init, Fore
import time

init(autoreset=True)

BASE_URL = "https://api.binance.com/api/v3/klines"
PRICE_URL = "https://api.binance.com/api/v3/ticker/24hr"

# 🔹 Apna Discord webhook URL yahan paste karo
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1411968945476337786/kMj3PfIomoBv9xWPg72LXKldILJpgEptlP0wT7m85yxJ9z1UsDDF-ePTq7Log9nNhF2M"

# Timeframes
timeframes = {
    "15m": "15m",
    "1h": "1h",
    "4h": "4h"
}

# Discord par message bhejne ka function
def send_discord_message(message):
    try:
        data = {"content": message}
        response = requests.post(DISCORD_WEBHOOK_URL, json=data)
        if response.status_code == 204:
            print("✅ Discord par message bheja gaya!")
        else:
            print(f"❌ Discord Error: {response.status_code}, {response.text}")
    except Exception as e:
        print(f"⚠️ Discord error: {e}")

# Fetch USDT pairs
def get_usdt_pairs():
    url = "https://api.binance.com/api/v3/exchangeInfo"
    data = requests.get(url).json()
    return [s['symbol'] for s in data['symbols'] if s['quoteAsset'] == "USDT" and s['status'] == "TRADING"]

# Fetch OHLCV data
def get_klines(symbol, interval="15m", limit=100):
    try:
        url = f"{BASE_URL}?symbol={symbol}&interval={interval}&limit={limit}"
        data = requests.get(url).json()
        df = pd.DataFrame(data, columns=[
            "time","open","high","low","close","volume",
            "close_time","qav","num_trades","tbbav","tbqav","ignore"
        ])
        df["open"] = df["open"].astype(float)
        df["high"] = df["high"].astype(float)
        df["low"] = df["low"].astype(float)
        df["close"] = df["close"].astype(float)
        df["volume"] = df["volume"].astype(float)

        # Indicators
        df["ema7"] = df["close"].ewm(span=7).mean()
        df["sma21"] = df["close"].rolling(window=21).mean()
        df["rsi"] = compute_rsi(df["close"], 14)

        return df
    except Exception as e:
        print(f"⚠️ get_klines error {symbol}: {e}")
        return None

# RSI
def compute_rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=period-1, adjust=False).mean()
    ema_down = down.ewm(com=period-1, adjust=False).mean()
    rs = ema_up / ema_down
    return 100 - (100 / (1 + rs))

# Support & Resistance
def find_support_resistance(df):
    closes = df["close"].values
    support = np.min(closes[-20:])
    resistance = np.max(closes[-20:])
    return support, resistance

# Candlestick Pattern Detector
def detect_patterns(df):
    patterns = []
    o, h, l, c = df["open"].iloc[-1], df["high"].iloc[-1], df["low"].iloc[-1], df["close"].iloc[-1]
    prev_o, prev_c = df["open"].iloc[-2], df["close"].iloc[-2]

    body = abs(c - o)
    range_ = h - l

    # Doji
    if range_ > 0 and body / range_ < 0.1:
        patterns.append("Doji")

    # Hammer
    if body < (h - l) * 0.3 and (o - l) > body * 2 and (c - l) > body * 2:
        patterns.append("Hammer")

    # Shooting Star
    if body < (h - l) * 0.3 and (h - o) > body * 2 and (h - c) > body * 2:
        patterns.append("Shooting Star")

    # Bullish Engulfing
    if prev_c < prev_o and c > o and c > prev_o and o < prev_c:
        patterns.append("Bullish Engulfing")

    # Bearish Engulfing
    if prev_c > prev_o and c < o and c < prev_o and o > prev_c:
        patterns.append("Bearish Engulfing")

    # Morning Star
    if prev_c < prev_o and abs(o - c) < body * 0.3 and c > prev_o:
        patterns.append("Morning Star")

    # Evening Star
    if prev_c > prev_o and abs(o - c) < body * 0.3 and c < prev_o:
        patterns.append("Evening Star")

    return patterns

# Apply strategy
def apply_strategy(df):
    close = df["close"].iloc[-1]
    volume = df["volume"].iloc[-1]
    prev_volume = df["volume"].iloc[-2]
    ema = df["ema7"].iloc[-1]
    sma = df["sma21"].iloc[-1]
    rsi = df["rsi"].iloc[-1]
    support, resistance = find_support_resistance(df)
    patterns = detect_patterns(df)

    # Filtering logic
    strong_signal = False
    reasons = []

    if rsi < 30:
        strong_signal = True
        reasons.append("RSI Oversold (<30)")
    if rsi > 70:
        strong_signal = True
        reasons.append("RSI Overbought (>70)")
    if ema > sma:
        reasons.append("Bullish Crossover (EMA > SMA)")
    if ema < sma:
        reasons.append("Bearish Crossover (EMA < SMA)")
    if patterns:
        strong_signal = True
        reasons.extend(patterns)

    return {
        "price": close,
        "volume": volume,
        "prev_volume": prev_volume,
        "ema": ema,
        "sma": sma,
        "rsi": rsi,
        "support": support,
        "resistance": resistance,
        "patterns": patterns,
        "strong": strong_signal,
        "reasons": reasons
    }

# 🔹 Top Movers (24h change %)
def get_top_movers(limit=10):
    try:
        data = requests.get(PRICE_URL).json()
        df = pd.DataFrame(data)
        df = df[df["symbol"].str.endswith("USDT")]
        df["priceChangePercent"] = df["priceChangePercent"].astype(float)
        df = df.sort_values("priceChangePercent", ascending=False)
        top_gainers = df.head(limit)[["symbol","lastPrice","priceChangePercent"]]
        top_losers = df.tail(limit)[["symbol","lastPrice","priceChangePercent"]]
        return top_gainers, top_losers
    except Exception as e:
        print(f"⚠️ Top movers error: {e}")
        return None, None

# Run scan
def run_signal_scan():
    usdt_pairs = get_usdt_pairs()
    found = 0

    # Top 150 coins only
    top_pairs = usdt_pairs[:150]

    for symbol in top_pairs:
        confirmations = {}

        for tf_name, tf_interval in timeframes.items():
            try:
                df = get_klines(symbol, interval=tf_interval, limit=50)
                if df is None or len(df) < 25:
                    continue

                signal = apply_strategy(df)
                confirmations[tf_name] = signal

            except Exception as e:
                print(f"⚠️ Error for {symbol} at {tf_name} timeframe: {e}")
                continue
            time.sleep(0.05)

        # Multi-timeframe confirmation
        if all(confirmations.values()):
            strong_tfs = [tf for tf, sig in confirmations.items() if sig["strong"]]
            if len(strong_tfs) >= 2:  # At least 2 TF agree
                last_signal = confirmations["15m"]  # base on 15m
                arrow = "🔼" if last_signal['volume'] > last_signal['prev_volume'] else "🔽"

                msg = (
                    f"🚨 **{symbol} Signal**\n"
                    f"Confirmed on: {', '.join(strong_tfs)}\n"
                    f"Reasons: {', '.join(last_signal['reasons'])}\n"
                    f"Price: {last_signal['price']:.4f}\n"
                    f"Volume: {last_signal['volume']} {arrow}\n"
                    f"RSI: {last_signal['rsi']:.2f}\n"
                    f"EMA(7): {last_signal['ema']:.4f} | SMA(21): {last_signal['sma']:.4f}\n"
                    f"Support: {last_signal['support']:.4f} | Resistance: {last_signal['resistance']:.4f}"
                )

                print(msg)
                send_discord_message(msg)
                found += 1

    if found == 0:
        print("No strong signals found this round.")

    # 🔹 Send Top Movers once every scan
    gainers, losers = get_top_movers(limit=5)
    if gainers is not None:
        gm = "**📈 Top Gainers (24h)**\n" + "\n".join(
            [f"{row['symbol']}: {row['lastPrice']} ({row['priceChangePercent']}%)" for _, row in gainers.iterrows()]
        )
        lm = "**📉 Top Losers (24h)**\n" + "\n".join(
            [f"{row['symbol']}: {row['lastPrice']} ({row['priceChangePercent']}%)" for _, row in losers.iterrows()]
        )
        send_discord_message(gm + "\n\n" + lm)


if __name__ == "__main__":
    while True:
        run_signal_scan()
        time.sleep(15)   # refresh every 15 seconds


🚨 **ONTUSDT Signal**
Confirmed on: 15m, 1h, 4h
Reasons: Bearish Crossover (EMA < SMA), Shooting Star
Price: 0.1422
Volume: 36689.0 🔽
RSI: 34.07
EMA(7): 0.1423 | SMA(21): 0.1430
Support: 0.1415 | Resistance: 0.1444
✅ Discord par message bheja gaya!
🚨 **TRXUSDT Signal**
Confirmed on: 15m, 1h, 4h
Reasons: Bearish Crossover (EMA < SMA), Doji, Hammer, Shooting Star
Price: 0.3381
Volume: 1938902.9 🔼
RSI: 43.31
EMA(7): 0.3378 | SMA(21): 0.3379
Support: 0.3370 | Resistance: 0.3384
✅ Discord par message bheja gaya!
🚨 **USDCUSDT Signal**
Confirmed on: 15m, 1h, 4h
Reasons: Bullish Crossover (EMA > SMA), Doji, Hammer
Price: 0.9999
Volume: 1493040.0 🔽
RSI: 53.98
EMA(7): 0.9999 | SMA(21): 0.9998
Support: 0.9997 | Resistance: 0.9999
✅ Discord par message bheja gaya!
🚨 **LINKUSDT Signal**
Confirmed on: 15m, 1h, 4h
Reasons: Bearish Crossover (EMA < SMA), Shooting Star
Price: 22.9900
Volume: 13822.03 🔽
RSI: 43.93
EMA(7): 22.9305 | SMA(21): 22.9729
Support: 22.7700 | Resistance: 23.1300
✅ Discord par mes

KeyboardInterrupt: 